In [3]:
import random
import math
import sys
import sympy
random.seed(2)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
cases = [#'DUCT_1100',
         #'DUCT_1150',
         #'DUCT_1250',
         #'DUCT_1300',
         #'DUCT_1350',
         #'DUCT_1400',
         #'DUCT_1500',
         #'DUCT_1600',
         #'DUCT_1800',
         #'DUCT_2000',
         #'DUCT_2205',
         #'DUCT_2400',
         #'DUCT_2600',
         #'DUCT_2900',
         'DUCT_3200',
         #'DUCT_3500',
         ]

In [5]:
def loadCombinedArray(cases,field):
    data = np.concatenate([np.load('C:/Users/li/Desktop/ML-Turbulence-dataset/'+dataset+'/'+dataset+'_'+case+'_'+field + '.npy') for case in cases])
    return data

def loadLabels(cases,field):
    data = np.concatenate([np.load('C:/Users/li/Desktop/ML-Turbulence-dataset/'+'labels/'+case+'_'+field + '.npy') for case in cases])
    return data

In [6]:
dataset = 'komega'
Tensors = loadCombinedArray(cases,'Tensors')
print('Shape of basis tensor array: '+str(Tensors.shape))

#Load the 47 invariants (N,47) used by Wu et al. "Physics-informed machine learning approach for augmenting turbulence models: A comprehensive framework" (2018)
Invariants = loadCombinedArray(cases,'I1')
print('Shape of invariant features array: '+str(Invariants.shape))

Labels = loadLabels(cases,'b')
Ani_tensor = []

for i in range(len(Labels)):
    a = np.delete(Labels[i].reshape((1,9)),[3,6,7],axis=1)
    Ani_tensor.append(a)
print('Shape of DNS/LES labels array: '+str(len(Ani_tensor)))
##需要选择前五个张量，和前两个不变量

Shape of basis tensor array: (9216, 10, 3, 3)
Shape of invariant features array: (9216, 47)
Shape of DNS/LES labels array: 9216


In [7]:
class Genome():
    '''对函数符运算进行定义、终结符定义'''

    def __init__(self):
        '''定义函数、终结符
        functions   保存函数 如：*，/，+，—
        terminals   保存终结符 如：a，b
        genicTerminals 保存连接基因的终结符 如：0，1，2
        '''
        self.functions = {"max_arity": 0}
        self.terminals = []
        self.genicTerminals = range(1)

    #定义 add ,sub ,mul ,div 函数的运算
    #进行div保护，当除数为小于1e-6时，将除数设置为1
    
    def add(inputs):
        val = np.add(inputs.pop(-1),inputs.pop(-1))
        inputs.append(val)
        return inputs
        
    def sub(inputs):
        val = np.subtract(inputs.pop(-1),inputs.pop(-1))
        inputs.append(val)
        return inputs
    
    def mul(inputs):
        val = inputs.pop(-1)*inputs.pop(-1)
        inputs.append(val)
        return inputs
                      
    def sin(inputs):
        val = np.sin(inputs.pop(-1))
        inputs.append(val)
        return inputs
    
    def cos(inputs):
        val = np.cos(inputs.pop(-1))
        inputs.append(val)
        return inputs
    
    def exp(inputs):
        val = np.exp(inputs.pop(-1))
        inputs.append(val)
        return inputs
    
    def p(inputs,plasmid):
        val = np.multiply(inputs.pop(-1),plasmid)
        inputs.append(val)
        return inputs
        
    #运算符特征【'+':2,'-':2,'a':0,'b':0】
    def symbols(self):
        symbols = self.functions.copy()
        symbols.pop("max_arity", None)
        for symbol in symbols.keys():
            symbols[symbol] = symbols[symbol][1]
        symbols.update({terminal: 0 for terminal in self.terminals + list(self.genicTerminals)})
        return symbols

    '''传统基因函数集合、连接基因函数集合：
    对于：{"+": (add, 2),"max_arity": 2}
    +是运算符，add保存是对应的运算，2是需要参数个数
    max_arity 是保存最大需要的参数个数，2是最大需要参数个数'''

    gene_set = {"+": (add, 2), "-": (sub, 2), "*": (mul, 2), "s":(sin,1),"c":(cos,1),"p":(p,1),"max_arity": 2}
    plasmid_set = {"+": (add, 2), "-": (sub, 2), "*": (mul, 2), "s":(sin,1),"c":(cos,1),"max_arity": 2}

In [8]:
class Gene():
    def __init__(self,plasmid,genome):
        '''
        genome:         基因的 Genome
        head            基因头部
        tail            基因尾部
        '''
        self.plasmid = plasmid
        self.genome = genome
        self.Ghead = []
        self.Gtail = []
        self.Phead = []
        self.Ptail = []
    
    def initRand(self, headLength):
        """, numGenes
        """
        '''
        :param headLength:      基因头部长度
        :param numGenes:        基因个数
        :return:self

        max_arity       保存最大需要的参数个数
        functions       保存Genome中操作符 如:["+","-","*","/"]
        '''
        max_arity = self.plasmid.functions["max_arity"]
        
        Pfunctions = list(self.plasmid.functions.keys())
        Gfunctions = list(self.genome.functions.keys())
        
        Pfunctions.pop(Pfunctions.index("max_arity")) # functions中剔除max_arity
        Gfunctions.pop(Gfunctions.index("max_arity")) # functions中剔除max_arity

        Pterminals = self.plasmid.terminals
        Gterminals = self.genome.terminals
        
        self.Phead = [random.choice(Pfunctions + Pterminals) for i in range(headLength)]
        self.Ptail = [random.choice(Pterminals) for i in range(headLength * (max_arity - 1) + 1)]
        
        self.Ghead = [random.choice(Gfunctions + Gterminals) for i in range(headLength)]
        self.Gtail = [random.choice(Gterminals) for i in range(headLength * (max_arity - 1) + 1)]
        
        #基因首个为运算符
        self.Phead[0] = random.choice(Pfunctions)
        self.Ghead[0] = random.choice(Gfunctions)
        return self

    #例子【'+':2,'-':2,'*':2,'a':0,'a':0,'a':0,'a':0】
    #首先，取出一个符号'+',即arity初始为：arity=1
    #然后，保存该符号的arity数值。由'+':2得arity=2，同时将取出得'+'在elementLayers保存
    #循环，此时arity=2，取出2个符号即'-','*'。
    #根据'-','*'得到max_arity数值，max_arity=4
    #以此类推直到max_arity=0。
    #这其实就模拟树，一层一层得取出。
    #【【'+':2】，【'-':2,'*':2】,【'a':0,'a':0,'a':0,'a':0】】
    def orderPStack(self):
        elementLayers = []
        plasmid = self.Phead + self.Ptail
        arity = 1
        while arity > 0:
            elementList = []
            for i in range(arity):
                elementList.append(plasmid.pop(0))
            arity = 0
            for i_element in elementList:
                arity += self.plasmid.symbols()[i_element]
            elementLayers.append(elementList)
        # print(elementLayers)
        return elementLayers
    
    def orderGStack(self):
        elementLayers = []
        gene = self.Ghead + self.Gtail
        arity = 1
        while arity > 0:
            elementList = []
            for i in range(arity):
                elementList.append(gene.pop(0))
            arity = 0
            for i_element in elementList:
                arity += self.genome.symbols()[i_element]
            elementLayers.append(elementList)
        # print(elementLayers)
        return elementLayers
    
    #用递归实现逆波兰式的装转变
    
    def PtranformRPN(self, elementLayers, layerIndex = 0):
        '''
        :param elementLayers:    orderStack输出的层次结构
        :param layerIndex:       几层用于递归调用
        :return:
        '''
        stack = []
        for i in range(self.plasmid.symbols()[elementLayers[layerIndex][0]]):
            stack =  stack + self.PtranformRPN(elementLayers, layerIndex + 1)
            ##print(stack)
        stack.append(elementLayers[layerIndex].pop(0))
        #print(stack)
        return stack
    
    
    def GtranformRPN(self, elementLayers, layerIndex = 0):
        '''
        :param elementLayers:    orderStack输出的层次结构
        :param layerIndex:       几层用于递归调用
        :return:
        '''
        stack = []
        for i in range(self.genome.symbols()[elementLayers[layerIndex][0]]):
            stack =  stack + self.GtranformRPN(elementLayers, layerIndex + 1)
            ##print(stack)
        stack.append(elementLayers[layerIndex].pop(0))
        #print(stack)
        return stack

    #inputs=['a':1,'b':3]已赋值，取值运算
    def evalPstack(self, stack, inputs): 
        """plasmid
        """
        '''
        :param stack:       为逆波兰式
        :param inputs:      参数输入值
        :return:returnStack     保存最后的运算结果
        '''
        returnStack = []
        for symbol in stack:
            if symbol in inputs.keys():
                returnStack.append(inputs[symbol])
                #print('1',returnStack)
            else:
                self.plasmid.functions[symbol][0](returnStack)
                #print('2',returnStack)
        return returnStack[0]
    
    def evalGstack(self,Pstack,Pinputs,Gstack,Ginputs):
        plasmid = self.evalPstack(Pstack,Pinputs)
        returnStack = []
        for symbol in Gstack:
            if symbol in Ginputs.keys():
                returnStack.append(Ginputs[symbol])
                #print('1',returnStack)
            elif symbol == 'p':
                self.genome.functions[symbol][0](returnStack,plasmid)
            else:
                self.genome.functions[symbol][0](returnStack)
                #print('2',returnStack)
        return returnStack[0]

    def eval(self,Pinputs,Ginputs):
        '''
        对精英进行赋值运算评估
        :param inputs:      参数输入值
        :return:
        '''
        PelementLayers = self.orderPStack()
        GelementLayers = self.orderGStack()
        evalPStack = self.PtranformRPN(PelementLayers)
        evalGStack = self.GtranformRPN(GelementLayers)
        value = self.evalGstack(evalPStack,Pinputs,evalGStack,Ginputs)
        value = np.delete(value.reshape((1,9)),[3,6,7],axis=1)
        return value
    def replicate(self):
        '''
        创建新的端点，并对端点内的元素进行复制
        :return: newPlasmid    返回新的基因
        '''
        newPG = Gene(self.plasmid,self.genome)
        newPG.Phead = self.Phead[:]
        newPG.Ptail = self.Ptail[:]
        newPG.Ghead = self.Ghead[:]
        newPG.Gtail = self.Gtail[:]
        return newPG

In [9]:
class Chromosome():
    def __init__(self):
        '''
        gene           基因：用于保存传统基因和端粒
        '''
        self.genes = []
    #染色体（基因、连接函数）
    def initRand(self,numGenes,headLength,plasmid,genome):
        '''
        initialization
        :param numGenes:        Number of genes
        :param headLength:      Length of head
        :param plasmid:         plasmid Genome
        :param genome:          gene Genome
        :return self:
        genes           基因：用于保存传统基因
        '''
        self.genes = [Gene(plasmid,genome).initRand(headLength) for i in range(numGenes)]
        return self

        # 染色体评估

    def eval(self,Pinputs,Ginputs):
        '''
        :param inputs:      样本输入值
        :return:            returnValues
        '''
        returnValues = []
        values = []
        #基因赋值运算、结果保存在returnValues
        #将终结值赋予对应的数值,如 a:1 ,b:2
        for i in range(500):
            for gene in self.genes:
                evalPInputs = {terminal:value for (terminal,value) in list(zip(gene.plasmid.terminals,Pinputs[i]))}
                evalGInputs = {terminal: value for (terminal, value) in list(zip(gene.genome.terminals, Ginputs[i]))}
                returnValues.append(gene.eval(evalPInputs,evalGInputs))
            sum=0
            for value in returnValues:
                sum+=value
            values.append(sum)        
        #连接函数运算、最终结果保存在Values
        return values

    def replicate(self):
        '''
        基因复制
        newGene        保存新的基因
        newChromosome   创建新的染色体，并赋值
        :return: newChromosome 返回新的染色体
        '''
        newGene = []
        # 基因复制
        for gene in self.genes:
            newGene.append(gene.replicate())
        # 创建新的染色体
        newChromosome = Chromosome()
        newChromosome.genes = newGene
        return newChromosome

    def mutation(self, rate):
        '''
        :param rate:            基因突变的概率
        :return:
        head：           保存基因头部
        tail            保存基因尾部
        functions       保存操作符如：【“+”，“-”，“*”，“/”】
        '''
        for gene in self.genes:
            Phead = gene.Phead
            Ptail = gene.Ptail
            Ghead = gene.Ghead
            Gtail = gene.Gtail
            
            Pfunctions = list(gene.plasmid.functions.keys())
            Gfunctions = list(gene.genome.functions.keys())
            
            Pfunctions.pop(Pfunctions.index("max_arity"))
            Gfunctions.pop(Gfunctions.index("max_arity"))
            # 基因头部突变
            for i in range(len(Ghead)):
                if random.random() < rate:
                    Phead[i] = random.choice(Pfunctions + gene.plasmid.terminals)
                    Ghead[i] = random.choice(Gfunctions + gene.genome.terminals)

            # 基因尾部突变
            for i in range(len(Gtail)):
                if random.random() < rate:
                    Ptail[i] = random.choice(gene.plasmid.terminals)
                    Gtail[i] = random.choice(gene.genome.terminals)
    
    def ISTransposition(self, rate):
        '''
        IS转座是从种群中随机选择一个染色体，然后随机从IS转座长度中选择IS长度，
        然后在染色体中选择IS长度的基因片段，并随机选择基因，插入到除基因首元素之外的头部部分中。
        :param rate:            基因IS概率
        :return:
        seq             将所有基因进行连接保存
        start           选择一小段的起点
        stop            选择一小段的终点
        ISSeq           保存选出的小段
        insertPoint     插入点
        '''
        # 出现IS转座
        if random.random() < rate:
            Pseq = []
            Gseq = []
            for gene in self.genes:
                Pseq = Pseq + gene.Phead + gene.Ptail
                Gseq = Gseq + gene.Ghead + gene.Gtail

            #   随机选择一个基因
            gene = random.choice(self.genes)
            headLength = len(gene.Ghead)

            #   选择截取一小断保存在 ISSeq 中
            Pstart = random.randint(0, len(Pseq) - 1)
            Pstop = random.randint(Pstart, Pstart + headLength)
            Gstart = random.randint(0, len(Gseq) - 1)
            Gstop = random.randint(Gstart, Gstart + headLength)
            
            IS_P_Seq = Pseq[Pstart:Pstop]
            IS_G_Seq = Gseq[Gstart:Gstop]

            #   选择一个插入点，进行插入，并且去掉超过长度的部分
            PinsertPoint = random.randint(1, headLength - 1)
            GinsertPoint = random.randint(1, headLength - 1)
            
            for i in range(len(IS_P_Seq)):
                gene.Phead.insert(PinsertPoint + i, IS_P_Seq[i])
            gene.Phead = gene.Phead[:headLength]
            
            for i in range(len(IS_G_Seq)):
                gene.Ghead.insert(GinsertPoint + i, IS_G_Seq[i])
            gene.Ghead = gene.Ghead[:headLength]

    def RISTransposition(self, rate):
        '''
        :param rate:            基因RIS转座概率
        :return:
        seq             将所有基因进行连接保存
        functions       保存操作符如：【“+”，“-”，“*”，“/”】
        start           选择一小段的起点
        end             选择一小段的终点
        RISSeq           保存选出的小段
        insertPoint     插入点
        '''
        # 进行RIS转座
        if random.random() < rate:
            Pseq = []
            Gseq = []
            for gene in self.genes:
                Pseq = Pseq + gene.Phead + gene.Ptail
                Gseq = Gseq + gene.Ghead + gene.Gtail

            #   随机选择一个基因
            gene = random.choice(self.genes)
            headLength = len(gene.Ghead)
            Pfunctions = list(gene.plasmid.functions.keys())
            Pfunctions.pop(Pfunctions.index("max_arity"))
            Gfunctions = list(gene.genome.functions.keys())
            Gfunctions.pop(Gfunctions.index("max_arity"))

            #   选择截取一小断保存在 RISSeq 中
            Pstart = random.randint(0, len(Pseq) - 1)
            Gstart = random.randint(0, len(Gseq) - 1)
            #直到发现一个函数为止
            
            while Pseq[Pstart] not in Pfunctions:
                if Pstart == len(Pseq) - 1:
                    break
                Pstart += 1
            Pstop = random.randint(Pstart, Pstart + headLength)
            RIS_P_Seq = Pseq[Pstart:Pstop]
            
            while Gseq[Gstart] not in Gfunctions:
                if Gstart == len(Gseq) - 1:
                    break
                Gstart += 1
            Gstop = random.randint(Gstart, Gstart + headLength)
            RIS_G_Seq = Gseq[Gstart:Gstop]

            #进行插入，并且去掉超过长度的部分
            for i in range(len(RIS_P_Seq)):
                gene.Phead.insert(i,RIS_P_Seq[i])
            for i in range(len(RIS_G_Seq)):
                gene.Ghead.insert(i,RIS_G_Seq[i])
            
            gene.Phead = gene.Phead[:headLength]    
            gene.Ghead = gene.Ghead[:headLength]

    def geneTransposition(self, rate):
        '''
        基因转座仅仅改变基因在同一染色体中的位置
        :param rate:            基因转座概率
        :return:
        '''
        if random.random() < rate:
            self.genes[random.randint(0,len(self.genes)-1)] = random.choice(self.genes).replicate()


    def onePointRecombination(self, rate, otherChromosome):
        '''
        进行单点重组的时候，父代染色体相互配对并在相同的位置切断，两个染色体相互交换重组点之后的部分
        :param rate:                单点重组概率
        :param otherChromosome:     染色体
        :return:
        seq             当前染色体将所有基因进行连接保存
        otherSeq        不同于当前染色体将所有基因进行连接保存
        recombinationPoint       重组结点
        '''
        # 进行单点重组
        if random.random() < rate:
            Pseq = []
            Gseq = []
            otherPSeq = []
            otherGSeq = []
            for gene in self.genes:
                Pseq = Pseq + gene.Phead + gene.Ptail
                Gseq = Gseq + gene.Ghead + gene.Gtail
            for otherGene in otherChromosome.genes:
                otherPSeq = otherPSeq + otherGene.Phead + otherGene.Ptail
                otherGSeq = otherGSeq + otherGene.Ghead + otherGene.Gtail
            if len(Gseq) != len(otherGSeq):
                return
            # 进行单点重组（交换）
            PrecombinationPoint = random.randint(0, len(Pseq) - 1)
            Pseq[PrecombinationPoint:], otherPSeq[PrecombinationPoint:] = otherPSeq[PrecombinationPoint:], Pseq[PrecombinationPoint:]
            GrecombinationPoint = random.randint(0, len(Gseq) - 1)
            Gseq[GrecombinationPoint:], otherGSeq[GrecombinationPoint:] = otherGSeq[GrecombinationPoint:], Gseq[GrecombinationPoint:]

            # 单点重组后,当前染色体进行保存
            for gene in self.genes:
                gene.Phead = Pseq[:len(gene.Phead)]
                gene.Ghead = Gseq[:len(gene.Ghead)]
                del Pseq[0:len(gene.Phead)]
                del Gseq[0:len(gene.Ghead)]
                gene.Ptail = Pseq[:len(gene.Ptail)]
                gene.Gtail = Gseq[:len(gene.Gtail)]
                del Pseq[0:len(gene.Ptail)]
                del Gseq[0:len(gene.Gtail)]
                
            # 单点重组后,不同于当前染色体进行保存
            for otherGene in otherChromosome.genes:
                otherGene.Phead = otherPSeq[:len(otherGene.Phead)]
                otherGene.Ghead = otherGSeq[:len(otherGene.Ghead)]
                del otherPSeq[0:len(otherGene.Phead)]
                del otherGSeq[0:len(otherGene.Ghead)]
                otherGene.Ptail = otherPSeq[:len(otherGene.Ptail)]
                otherGene.Gtail = otherGSeq[:len(otherGene.Gtail)]
                del otherPSeq[0:len(otherGene.Ptail)]
                del otherGSeq[0:len(otherGene.Gtail)]


    def twoPointRecombination(self, rate, otherChromosome):
        '''
        进行两点重组的时候，父代染色体相互配对，在染色体中随机选择两个点，将染色体切断。
        两个染色体相互交换重组点之间的部分，形成两个新的子代染色体
        :param rate:                两点重组概率

        :param otherChromosome:     不同于当前的染色体
        :return:
        seq             当前染色体将所有基因进行连接保存
        otherSeq        不同于当前染色体将所有基因进行连接保存
        recombinationPoint       重组结点
        otherPoint               另一个重组结点
        '''
        # 进行两点重组
        if random.random() < rate:
            Pseq = []
            Gseq = []
            otherPSeq = []
            otherGSeq = []
            for gene in self.genes:
                Pseq = Pseq + gene.Phead + gene.Ptail
                Gseq = Gseq + gene.Ghead + gene.Gtail
            for otherGene in otherChromosome.genes:
                otherPSeq = otherPSeq + otherGene.Phead + otherGene.Ptail
                otherGSeq = otherGSeq + otherGene.Ghead + otherGene.Gtail
            if len(Pseq) != len(otherPSeq):
                return
            # 进行两点重组（交换）
            PrecombinationPoint = random.randint(0, len(Pseq) - 1)
            PotherPoint = random.randint(PrecombinationPoint, len(Pseq) - 1)
            Pseq[PrecombinationPoint:], otherPSeq[PrecombinationPoint:] = otherPSeq[PrecombinationPoint:], Pseq[PrecombinationPoint:]
            Pseq[:PotherPoint], otherPSeq[:PotherPoint] = otherPSeq[:PotherPoint], Pseq[:PotherPoint]
            
            GrecombinationPoint = random.randint(0, len(Gseq) - 1)
            GotherPoint = random.randint(GrecombinationPoint, len(Gseq) - 1)
            Gseq[GrecombinationPoint:], otherGSeq[GrecombinationPoint:] = otherGSeq[GrecombinationPoint:], Gseq[GrecombinationPoint:]
            Gseq[:GotherPoint], otherGSeq[:GotherPoint] = otherGSeq[:GotherPoint], Gseq[:GotherPoint]
            # 两点重组后,当前染色体进行保存
            for gene in self.genes:
                gene.Phead = Pseq[:len(gene.Phead)]
                gene.Ghead = Gseq[:len(gene.Ghead)]
                del Pseq[0:len(gene.Phead)]
                del Gseq[0:len(gene.Ghead)]
                gene.Ptail = Pseq[:len(gene.Ptail)]
                gene.Gtail = Gseq[:len(gene.Gtail)]
                del Pseq[0:len(gene.Ptail)]
                del Gseq[0:len(gene.Gtail)]
                
            # 两点重组后,当前染色体进行保存
            for otherGene in otherChromosome.genes:
                otherGene.Phead = otherPSeq[:len(otherGene.Phead)]
                otherGene.Ghead = otherGSeq[:len(otherGene.Ghead)]
                del otherPSeq[0:len(otherGene.Phead)]
                del otherGSeq[0:len(otherGene.Ghead)]
                otherGene.Ptail = otherPSeq[:len(otherGene.Ptail)]
                otherGene.Gtail = otherGSeq[:len(otherGene.Gtail)]
                del otherPSeq[0:len(otherGene.Ptail)]
                del otherGSeq[0:len(otherGene.Gtail)]
                

    def geneRecombination(self, rate, otherChromosome):
        '''
        在 GEP 的第三种重组中，两个染色体中的整个基因相互交换，
        形成的两个子代染色体含有来自两个父体的基因。
        :param rate:            基因重组概率
        :param otherChromosome: 不同于当前的染色体
        :return:
        '''
        # 进行基因重组
        if random.random() < rate:
            if len(self.genes) != len(otherChromosome.genes):
                return
            recombinationPoint = random.randint(0, len(self.genes) - 1)
            self.genes[recombinationPoint:recombinationPoint+1], otherChromosome.genes[recombinationPoint:recombinationPoint+1] = otherChromosome.genes[recombinationPoint:recombinationPoint+1], self.genes[recombinationPoint:recombinationPoint+1]


    def printChromosome(self):
        for gene in self.genes:
            print('Plasmid:',gene.Phead + gene.Ptail,end=':\n')
            print('Genome:',gene.Ghead + gene.Gtail, end=':\n')
        print()

In [10]:
genome = Genome()
plasmid = Genome()
genome.functions.update(Genome.gene_set)
plasmid.functions.update(Genome.plasmid_set)
plasmid.terminals = ['l1','l2']
genome.terminals = ['T1','T2','T3','T4','T5']

In [11]:
gene = Gene(plasmid,genome)
gene.initRand(4)

In [12]:
chor = Chromosome()

In [13]:
chor.initRand(2,3,plasmid,genome)

In [14]:
chor.printChromosome()

Plasmid: ['-', 'c', '*', 'l1', 'l1', 'l2', 'l2']:
Genome: ['-', 'T1', 'T1', 'T5', 'T2', 'T5', 'T2']:
Plasmid: ['s', '-', '*', 'l1', 'l1', 'l2', 'l1']:
Genome: ['-', 'T1', 'T3', 'T3', 'T5', 'T3', 'T3']:



In [15]:
test_value = chor.eval(Invariants,Tensors)

In [16]:
class Environment:
    def __init__(self):
        '''
        population          种群：用于保存染色体
        bestChromosome      最佳染色体：保存当代最佳染色体
        bestfitness         最佳染色体的适应度：保证当代最佳染色体的适应度
        mutationRate        突变
        ISTranspositionRate         IS
        RISTranspositionRate        RIS
        geneTranspositionRate       基因转座
        onePointRecombinationRate   单点重组
        twoPointRecombinationRate   两点重组
        geneRecombination           基因重组
        '''

        self.population = []
        self.bestChromosome=[]
        self.bestfitness=0
        self.mutationRate = 0
        self.ISTranspositionRate = 0
        self.RISTranspositionRate = 0
        self.geneTranspositionRate = 0
        self.onePointRecombinationRate = 0
        self.twoPointRecombinationRate = 0
        self.geneRecombination=0

    def setRates(self, mutationRate=0,ISTranspositionRate=0, RISTranspositionRate=0, geneTranspositionRate=0, onePointRecombinationRate=0, twoPointRecombinationRate=0, geneRecombinationRate=0):
        '''
        mutationRate        突变
        ISTranspositionRate         IS
        RISTranspositionRate        RIS
        geneTranspositionRate       基因转座
        onePointRecombinationRate   单点重组
        twoPointRecombinationRate   两点重组
        geneRecombination           基因重组
        '''
        self.mutationRate = mutationRate
        self.ISTranspositionRate = ISTranspositionRate
        self.RISTranspositionRate = RISTranspositionRate
        self.geneTranspositionRate = geneTranspositionRate
        self.onePointRecombinationRate = onePointRecombinationRate
        self.twoPointRecombinationRate = twoPointRecombinationRate
        self.geneRecombinationRate = geneRecombinationRate

    def init(self, populationSize, numGenes, headLength, plasmid, genome):
        '''
        初始化种群
        populationSize      种群大小
        numGenes            染色体基因个数
        headLength          基因头部长度
        genome              基因 Genome类
        '''
        self.population = [Chromosome().initRand(numGenes,headLength,plasmid,genome) for i in range(populationSize)]


    def run(self, Pinputs, Ginputs, Tvalues, evalFunction,generationCount=200,M=100,DataCount=10):
        '''
        :param inputsOutputs:   保存输入与输出集合
        :param evalFunction:    适应度函数
        :param generationCount: 执行代数
        :return:
        generation:             表示第几代种群
        i_chromosome            用于保存当代种群第i条染色体
        C_valueList             表示染色体,样本输出值
        fitness                 保存当前染色体的适应度
        maxFitness_flag=True    为True,返回的maxFitness:M*DataCount
        '''

        ResultBestFitness=float("-inf")
        ResultBestChromosome=[]

        generation = 0
        while True:

            sum_fitness=0
            NowBestFitness=float("-inf")
            NowBestChromosome=[]
            #进行种群打印
            print("generation: ",generation)
            # 可以打印每一染色出来看。。。
            self.printChromosomes(generation)
            generation += 1



            for i in range(len(self.population)):
                ############################# 这两个之间就是计算适应度函数的，其中计算的公式按照evalFunction
                #函数定义来计算的，结果fitness种保存，打印看下fitness=1640.5383020628487 表示这个染色体的适应度值
                #for i in range(len(self.population)) 每个染色都需要计算。适应度
                C_valueList = []
                i_chromosome = self.population[i]

                #计算每个染色体对应的 C_value
                # print(inputsOutputs[0])
                test_value = i_chromosome.eval(Pinputs, Ginputs)
                #evalFunction 进行适应度评估
                fitness = evalFunction(test_value, Tvalues, M=M, DataCount=DataCount)
                print(fitness)
                #############################
                # 这里将得到的适应度加起来 保存在sum_fitness
                sum_fitness=sum_fitness+fitness

                #这里 ：-》后边这句话就是了保存当代种群最佳适应度的值,与对应的染色体
                if fitness > NowBestFitness:
                    NowBestFitness = fitness
                    NowBestChromosome=i_chromosome

                # 完美解停止执行
                # 判断是否是完美解，就是跟最大适应度值还有多大差距，
                #abs(fitness - evalFunction([], [], maxFitness_flag=True,M=M,DataCount=DataCount))<1e-05:
                # 小于
                """"
                if abs(fitness - evalFunction([], [], maxFitness_flag=True,M=M,DataCount=DataCount))<1e-05:
                    print("*" * 46, "  完美解  ", "*" * 46)
                    i_chromosome.printChromosome()
                    return i_chromosome
                """
                ##-----输出测试-----##
                # print("[",i,"] ","   Fitness=",fitness)
                ####----------输出测试----------####
                self.population[i] = (i_chromosome, fitness)

            # 打印当代最佳染色体
            print("NowBestFitness=",NowBestFitness)
            NowBestChromosome.printChromosome()
            print("*"*46)

            # 保存最终最佳解，与对应的染色体
            if ResultBestFitness<NowBestFitness:
                ResultBestFitness=NowBestFitness
                ResultBestChromosome=NowBestChromosome

            # 繁衍代数大于generationCount退出, 并输出最佳解
            if generation >= generationCount:
                print("*"*46,"the best chromesome after",generationCount,"generation","*"*46)
                print("result_fitness=  ", ResultBestFitness)
                ResultBestChromosome.printChromosome()
                return ResultBestChromosome

            selected=self.select(sum_fitness,NowBestChromosome)
            self.replicate(selected)
            self.modify()
            self.population[0]=NowBestChromosome

    def select(self, sum_fitness,NowBestChromosome):
        '''
        :param sum_fitness:         总的适应度
        :param NowBestChromosome:   用于保存当代最佳染色体
        :return: selected           用于保存赌盘选择出的染色体
        :self.population[i] = (i_chromosome, fitness) 其中保存了染色体的适应度
        '''
        accumulator = 0  # 设置累加器
        roulette=[]     # 设置轮盘
        selected=[]

        #制作赌盘，如果percentage  区间为0的
        for i in range(len(self.population)):
            percentage = self.population[i][1] / sum_fitness * 10
            if percentage != 0:
                roulette.append(percentage + accumulator)
                accumulator = accumulator + percentage
            else:
                roulette.append(0)

        #一共选择 len(self.population)-1 条染色体体，最后一条保存最佳染色体。
        for i in range(len(self.population)-1):
            #产生随机数
            ranNumber=random.uniform(0,10)
            #进行选择染色体
            for i_pos in range(len(self.population)):
                if ranNumber <= roulette[i_pos]:
                    selected.append(self.population[i_pos][0])
                    break
        #最后一条保存最佳染色体
        selected.append(NowBestChromosome)
        return  selected

    def replicate(self, chromosomes):
        '''
        进行染色体复制，保证更新在 self.population
        :param chromosomes:       赌盘后选择种群染色体
        :return:
        '''
        self.population = []
        for chromosome in chromosomes:
            self.population.append(chromosome.replicate())

    def modify(self):
        '''
        染色体修饰
        mutation:           突变
        ISTransposition     IS转座
        RISTransposition    RIS转座
        geneTransposition   基因转座
        onePointRecombination   单点重组
        twoPointRecombination   两点重组
        geneRecombination       基因重组
        :return:
        '''
        for i in range(len(self.population)):
            chromosome = self.population[i]
            chromosome.mutation(self.mutationRate)
            chromosome.ISTransposition(self.ISTranspositionRate)
            chromosome.RISTransposition(self.RISTranspositionRate)
            chromosome.geneTransposition(self.geneTranspositionRate)
            #选择于i不同的染色体的，otherIndex
            otherIndex = i
            while otherIndex == i:
                otherIndex = random.randint(0, len(self.population) - 1)
            chromosome.onePointRecombination(self.onePointRecombinationRate, self.population[otherIndex])
            otherIndex = i
            while otherIndex == i:
                otherIndex = random.randint(0, len(self.population) - 1)
            chromosome.twoPointRecombination(self.twoPointRecombinationRate, self.population[otherIndex])
            otherIndex = i
            while otherIndex == i:
                otherIndex = random.randint(0, len(self.population) - 1)
            chromosome.geneRecombination(self.geneRecombinationRate, self.population[otherIndex])

    def printChromosomes(self, generation):
        '''
        打印染色体
        :param generation:   迭代的代数
        :return: None
        '''
        print("generation: ",generation)
        for i in range(len(self.population)):
            print(i, ':', end='')
            for gene in self.population[i].genes:
                print(gene.Phead + gene.Ptail, end=':\n')
                print(gene.Ghead + gene.Gtail, end=':\n')
            print()
        print()

In [17]:
def evalFunction(C_value, T_value, maxFitness_flag=False,M=100,DataCount=10):
    '''
    :param C_value:         表示染色体,样本输出值
    :param T_value:         表示染色体,样本真实值
    :param maxFitness_flag: 用于是否为最大适应度值
    :param M:               M 常数
    :param DataCount:       数据的个数
    :return:
    '''

    if maxFitness_flag:
        return M*DataCount
    sum_fitness = 0
    for i in range(500):
        for j in range(6):
            values = abs(C_value[i][0][j] - T_value[i][0][j])
        result = M - values
        if result < 0: result = 0
        sum_fitness = result + sum_fitness
    fitness = sum_fitness
    return fitness

In [18]:
envir = Environment()

In [19]:
envir.init(5,3,4,plasmid,genome)

In [20]:
envir.setRates(mutationRate=0.044,ISTranspositionRate=0.1,
                         RISTranspositionRate=0.1, geneTranspositionRate=0.1,onePointRecombinationRate=0.3,
                         twoPointRecombinationRate=0.3, geneRecombinationRate=0.1)

In [21]:
M=100
generationCount=100
DataCount = 500

In [22]:
result=envir.run(Invariants,Tensors,Ani_tensor,evalFunction,M=M,generationCount=generationCount,DataCount=DataCount)

generation:  0
generation:  0
0 :['s', 's', 'l1', 'l1', 'l2', 'l1', 'l2', 'l2', 'l2']:
['s', 'T3', 'p', 'T5', 'T4', 'T4', 'T3', 'T5', 'T5']:
['s', '-', '*', 'l2', 'l1', 'l2', 'l2', 'l2', 'l2']:
['c', 'T3', 'T3', 'T3', 'T5', 'T5', 'T4', 'T3', 'T2']:
['*', 'l1', 'c', '+', 'l2', 'l1', 'l1', 'l1', 'l1']:
['-', 'T5', '+', 'c', 'T5', 'T2', 'T1', 'T5', 'T2']:

1 :['*', '-', '+', 's', 'l1', 'l1', 'l2', 'l2', 'l1']:
['*', 'T5', '+', '-', 'T1', 'T1', 'T1', 'T1', 'T1']:
['+', 'l2', '-', 'l1', 'l1', 'l1', 'l2', 'l1', 'l1']:
['*', '+', '+', 'p', 'T5', 'T1', 'T3', 'T3', 'T4']:
['c', 'c', 'l2', 'c', 'l1', 'l2', 'l2', 'l1', 'l2']:
['c', '-', 'T5', 'T5', 'T3', 'T1', 'T1', 'T4', 'T2']:

2 :['-', 's', 's', 'c', 'l2', 'l1', 'l2', 'l2', 'l2']:
['-', 'T1', 'T5', '+', 'T5', 'T2', 'T1', 'T3', 'T1']:
['s', '+', 's', 'l1', 'l1', 'l1', 'l1', 'l1', 'l2']:
['*', 'c', '-', 'T4', 'T2', 'T5', 'T5', 'T3', 'T3']:
['-', 'l2', '+', '-', 'l1', 'l2', 'l2', 'l1', 'l1']:
['+', '-', 's', '-', 'T1', 'T1', 'T2', 'T2', 'T1']:

3

7221.189931962694
35099.53597533795
35099.53607587239
28427.552371213667
NowBestFitness= 35099.53607587239
Plasmid: ['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
Genome: ['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['s', '*', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
Genome: ['c', 'T3', 'T2', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
Genome: ['p', 'T5', 'T2', '-', 'T1', 'T1', 'T2', 'T1', 'T4']:

**********************************************
generation:  4
generation:  4
0 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
['s', '*', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
['c', 'T3', 'T2', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
['p', 'T5', 'T2', '-', 'T1', 'T1', 'T2', 'T1', 'T4']:

1 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T2', 'T1', 'T1', 'T2']:
['s', '*

5717.258760532132
7221.189944633942
35099.53607587239
NowBestFitness= 35099.53607587239
Plasmid: ['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
Genome: ['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['s', '*', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
Genome: ['c', 'T3', 'T2', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
Genome: ['p', 'T5', 'T2', '-', 'T1', 'T1', 'T2', 'T1', 'T4']:

**********************************************
generation:  8
generation:  8
0 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
['s', '*', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
['c', 'T3', 'T2', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
['p', 'T5', 'T2', '-', 'T1', 'T1', 'T2', 'T1', 'T4']:

1 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
['s', '*', 's', '*', 'l2', 

35099.536330964256
35099.536330964256
35099.536330964256
35099.53607587239
35099.53607685902
NowBestFitness= 35099.536330964256
Plasmid: ['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
Genome: ['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['s', 'l2', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
Genome: ['c', 'T4', 'p', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
Genome: ['p', '*', 'T2', '-', 'T1', 'T5', 'T2', 'T1', 'T4']:

**********************************************
generation:  12
generation:  12
0 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
['s', 'l2', 's', '*', 'l2', 'l2', 'l1', 'l2', 'l2']:
['c', 'T4', 'p', 'T1', 'T4', 'T3', 'T4', 'T2', 'T2']:
['-', 'c', 's', 'l1', 'l1', 'l1', 'l1', 'l2', 'l1']:
['p', '*', 'T2', '-', 'T1', 'T5', 'T2', 'T1', 'T4']:

1 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l2']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 

35099.536759213996
35099.536759213996
561.3203720625087
1055.410251092444
7221.190049664255
NowBestFitness= 35099.536759213996
Plasmid: ['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
Genome: ['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
Plasmid: ['s', 'l2', 's', 'l2', 'l2', 'l2', 'l1', 'l2', 'l2']:
Genome: ['c', 'T4', 'p', 'T1', 'T2', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['-', 'c', 's', '-', 'l1', 'l1', 'l1', 'l2', 'l1']:
Genome: ['p', '*', 'T2', '-', 'T1', 'T5', 'T2', 'T1', 'T4']:

**********************************************
generation:  16
generation:  16
0 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', 'T2', 'T2']:
['s', 'l2', 's', 'l2', 'l2', 'l2', 'l1', 'l2', 'l2']:
['c', 'T4', 'p', 'T1', 'T2', 'T1', 'T4', 'T1', 'T2']:
['-', 'c', 's', '-', 'l1', 'l1', 'l1', 'l2', 'l1']:
['p', '*', 'T2', '-', 'T1', 'T5', 'T2', 'T1', 'T4']:

1 :['s', '-', 'l2', '*', 'l2', 'l1', 'l2', 'l1', 'l1']:
['+', 'T4', 'p', 'T5', 'T4', 'T3', 'T4', '

...

49939.20036775624
3241.270302022391
49938.60771625914
1887.355033356743
49939.20036775624
NowBestFitness= 49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

**********************************************
generation:  84
generation:  84
0 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

1 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T

49939.20036775624
1038.2371752323086
35099.536233104096
NowBestFitness= 49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

**********************************************
generation:  88
generation:  88
0 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

1 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['+', '*', '*', '+', 'l2', 'l

2782.8927277216408
49939.20036775624
49939.20036775624
49938.6079489679
NowBestFitness= 49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

**********************************************
generation:  92
generation:  92
0 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

1 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T2', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', 'l1', '

7221.18995533953
310.14065040161336
49939.20036775624
49939.20036775624
NowBestFitness= 49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

**********************************************
generation:  96
generation:  96
0 :['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

1 :['-', '-', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
['*', '*', '*

49939.20036775624
49938.60771625914
49939.20036775624
49939.20036775624
NowBestFitness= 49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2', 's', 'T1', 'T5', 'T2', 'T1', 'T2']:

**********************************************
**********************************************    100 代后最佳解   **********************************************
result_fitness=   49939.20036775624
Plasmid: ['-', '+', '*', '*', 'l2', 'l2', 'l2', 'l1', 'l1']:
Genome: ['T4', 'T2', '-', 'T5', 'T4', 'T1', 'T4', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '+', 'l2', 'l2', 'l1', 'l2', 'l1']:
Genome: ['T3', 'c', '*', 'T5', 'T2', 'T5', 'T5', 'T1', 'T2']:
Plasmid: ['*', '*', '*', '*', 'l1', 'l1', 'l1', 'l1', 'l1']:
Genome: ['p', '*', 'T2',